<a href="https://colab.research.google.com/github/love-bees/twitter/blob/master/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy, csv

auth = tweepy.OAuthHandler('bzz', 
                           'bzz')

api = tweepy.API(auth, 
                 wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)

In [ ]:
new_tweets = 0

while True:
    try:
        tweet_ids = [int(tweet_id) for tweet_id in open('gathered_tweet_ids.txt', 'r').read().split("\n") if tweet_id.strip() != ""]

        csv_file = open("data.csv", "a")
        csv_writer = csv.writer(csv_file)

        for tweet in tweepy.Cursor(api.search, q = "covid", since_id = max(tweet_ids), tweet_mode = "extended", count = 100).items():
            if tweet.geo:
                if tweet.id not in tweet_ids:

                    csv_writer.writerow([tweet.id_str, 
                                         str(tweet.created_at),
                                         ", ".join([str(coordinate) for coordinate in tweet.coordinates["coordinates"][::-1]]), 
                                         tweet.lang, 
                                         " ".join([hashtag["text"] for hashtag in tweet.entities["hashtags"]]).encode("utf-8"), 
                                         tweet.full_text.encode('utf-8')])

                    with open('gathered_tweet_ids.txt', 'a') as file:
                        file.write("\n" + str(tweet.id))
                    tweet_ids.append(tweet.id)

                    print("Tweets collected: " + str(len(tweet_ids)))
                    csv_file.flush()

                    new_tweets += 1
                    
                    if new_tweets == 50:
                        new_tweets = 0
                        csv_file.close()
                        break
    except:

        continue